# Data Processing Code 
## Convergence Comparison of Optimization Objectives

In [ ]:
import numpy as np
from task import SUMO_task, pbounds
import matplotlib.pyplot as plt
import pickle
from multi_object_optimization import MooSUMOProblem, SinSUMOProblem
import pandas as pd
from matplotlib.font_manager import FontProperties
import itertools
from util import json2pd
%matplotlib inline

plt.rcParams['font.size'] = 15 
plt.rcParams['font.family'] = ['Times New Roman','SimSun']
plt.rcParams['font.weight'] = 'light'
plt.rcParams['axes.unicode_minus'] = False  
algo = ["pso", "nsga3", "age2"]
SAVED_PATH = "../output/plot/iteration"


def plot_object_f(env, algo_list, title=None):
    markers = itertools.cycle(('^', 'x', 's', '*', 'h', 'H', 'D', 'd'))
    marker_size = 6
    line_styles = itertools.cycle((':', '--', '-.', '-'))
    plt.figure(figsize=(4.0, 3.5))
    fig, ax = plt.subplots()
    
    min_points = []  # Store minimum value points for each algorithm
    lines = []       # Store plot line objects for each algorithm to get colors

    for algo in algo_list:
        with open(f'../output/data_cache/{env}_{algo}.pkl', 'rb') as f:
            res = pickle.load(f)
            history = res.history
            F_list = [entry.pop.get('F') for entry in history]
            all_F = np.vstack(F_list)
            df = pd.DataFrame(all_F, columns=[f'target{i}' for i in range(all_F.shape[1])])
            df['mean_targets'] = df.mean(axis=1)
            turning_points = df['mean_targets'].cummin()
            
            # Record minimum value point
            min_idx = turning_points.idxmin()
            min_val = turning_points.min()
            min_points.append((min_idx, min_val, algo.upper()))

            # Plot curve and store line object
            line, = plt.plot(df.index, turning_points, label=f'{algo.upper()}', marker=next(markers), 
                             markersize=marker_size, markerfacecolor='none', 
                             markevery=[i for i in range(1, len(turning_points) - 1) 
                                        if turning_points[i] != turning_points[i - 1] 
                                        or turning_points[i] != turning_points[i + 1]], 
                             linewidth=0.8)
            lines.append(line)

    # Add Bayesian algorithm results
    df = json2pd(f'../log/{env}.log')
    df["cummax_target"] = df["target"].cummax()
    bayesian_min_idx = (-df["cummax_target"]).idxmin()
    bayesian_min_val = (-df["cummax_target"]).min()
    min_points.append((bayesian_min_idx, bayesian_min_val, "Bayesian"))

    bayesian_line, = plt.plot(df.index, -df["cummax_target"], label="Bayesian", marker=next(markers), 
                              markerfacecolor='none', 
                              markevery=[i for i in range(1, len(df["cummax_target"]) - 1) 
                                         if df["cummax_target"].iloc[i] != df["cummax_target"].iloc[i - 1] 
                                         or df["cummax_target"].iloc[i] != df["cummax_target"].iloc[i + 1]], 
                              markersize=marker_size, linewidth=0.8)
    lines.append(bayesian_line)

    # Find global minimum point
    global_min = min(min_points, key=lambda x: x[1])
    
    # Add annotations and arrows for each algorithm's minimum value
    for i, (x, y, algo_name) in enumerate(min_points):
        arrow_color = lines[i].get_color()  # Dynamically get the color of the corresponding curve
        if (x, y) == (global_min[0], global_min[1]):  # Use different style for global minimum point
            plt.annotate(f'{y:.3f}',
                        xy=(x, y), xycoords='data',
                        xytext=(-5, 70), textcoords='offset points',
                        arrowprops=dict(arrowstyle="->", color=arrow_color,shrinkB=25, lw=1.),  
                        ha='center', va='bottom', fontweight='bold')
        else:
            plt.annotate(f'{y:.3f}',
                        xy=(x, y), xycoords='data',
                        xytext=(-5, 80), textcoords='offset points',
                        arrowprops=dict(arrowstyle="->", color=arrow_color, lw=1,shrinkB=25), 
                        ha='center', va='bottom')

    plt.ylim(0.05, 0.6)
    plt.xlim(-100, 3100)
    plt.legend(loc='upper right')
    plt.title(f'GoF Comparison in {title.upper()} Scenario')
    plt.xlabel('Iterations')
    plt.ylabel('Minimum KL Divergence')

    plt.tight_layout(pad=1)
    plt.grid(visible=True, linestyle='--', alpha=0.5)
    
    plt.savefig(f'{SAVED_PATH}/{env}_comparison.pdf', dpi=300)
    plt.show()


plot_object_f("merge", algo, "MERGE")
plot_object_f("right", algo, "NARROW")
plot_object_f("stop", algo, "STOP")


## Comparison of Convergence Performance of Different Algorithms in Various Scenarios $\LaTeX$ Table

In [ ]:
import numpy as np
import pickle
import pandas as pd
from util import json2pd
import os

def load_and_analyze_data_extended(env_list, algo_list):
    """
    Load and analyze the convergence performance of different algorithms in various scenarios, including extended statistics.
    
    Args:
    env_list: List of scenarios, e.g., ["merge", "right", "stop"]
    algo_list: List of algorithms, e.g., ["pso", "nsga3", "age2"]
    
    Returns:
    results_df: DataFrame containing the convergence performance of each algorithm in each scenario
    """
    results = []
    
    env_name_map = {
        "merge": "Merge",
        "right": "Narrow Road",
        "stop": "Stop and Give Way"
    }
    
    for env in env_list:
        try:
            bayesian_df = json2pd(f'../log/{env}.log')
            bayesian_df["cummax_target"] = bayesian_df["target"].cummax()
            bayesian_min_idx = (-bayesian_df["cummax_target"]).idxmin()
            bayesian_min_val = -bayesian_df["cummax_target"].min()
            bayesian_iterations = len(bayesian_df)
            initial_kl = -bayesian_df["cummax_target"].iloc[0]
            
            # 1. Convergence rate
            convergence_rate = (initial_kl - bayesian_min_val) / max(1, bayesian_min_idx)
            
            # 2. Stability metric (std of last 100 iterations)
            if len(bayesian_df) > 100:
                stability = bayesian_df["cummax_target"].iloc[-100:].std()
            else:
                stability = bayesian_df["cummax_target"].std()
            
            # 3. Initial convergence speed (first 100 iterations)
            early_iterations = min(100, len(bayesian_df))
            if early_iterations > 1:
                early_values = -bayesian_df["cummax_target"].iloc[:early_iterations]
                early_convergence = (early_values.iloc[0] - early_values.iloc[-1]) / early_iterations
            else:
                early_convergence = 0
            
            # 4. Relative improvement rate
            if initial_kl > 0:
                relative_improvement = (initial_kl - bayesian_min_val) / initial_kl * 100
            else:
                relative_improvement = 0
            
            results.append({
                "Scenario": env_name_map.get(env, env),
                "Algorithm": "Bayesian",
                "Min KL Divergence": bayesian_min_val,
                "Convergence Iterations": bayesian_min_idx,
                "Total Iterations": bayesian_iterations,
                "Convergence Efficiency": bayesian_min_idx / bayesian_iterations,
                "Convergence Rate": convergence_rate,
                "Stability Metric": stability,
                "Initial Convergence Speed": early_convergence,
                "Relative Improvement (%)": relative_improvement
            })
        except Exception as e:
            print(f"Failed to load Bayesian algorithm data in {env} scenario: {e}")
        
        # Process other algorithms
        for algo in algo_list:
            try:
                with open(f'../output/data_cache/{env}_{algo}.pkl', 'rb') as f:
                    res = pickle.load(f)
                    history = res.history
                    F_list = [entry.pop.get('F') for entry in history]
                    all_F = np.vstack(F_list)
                    df = pd.DataFrame(all_F, columns=[f'target{i}' for i in range(all_F.shape[1])])
                    df['mean_targets'] = df.mean(axis=1)
                    turning_points = df['mean_targets'].cummin()
                    
                    # Basic metrics
                    min_idx = turning_points.idxmin()
                    min_val = turning_points.min()
                    total_iterations = len(turning_points)
                    initial_kl = turning_points.iloc[0]
                    
                    # Extended metrics
                    # 1. Convergence rate
                    convergence_rate = (initial_kl - min_val) / max(1, min_idx)
                    
                    # 2. Stability metric (std of last 100 iterations)
                    if len(turning_points) > 100:
                        stability = turning_points.iloc[-100:].std()
                    else:
                        stability = turning_points.std()
                    
                    # 3. Initial convergence speed (first 100 iterations)
                    early_iterations = min(100, len(turning_points))
                    if early_iterations > 1:
                        early_values = turning_points.iloc[:early_iterations]
                        early_convergence = (early_values.iloc[0] - early_values.iloc[-1]) / early_iterations
                    else:
                        early_convergence = 0
                    
                    # 4. Relative improvement rate
                    if initial_kl > 0:
                        relative_improvement = (initial_kl - min_val) / initial_kl * 100
                    else:
                        relative_improvement = 0
                    
                    results.append({
                        "Scenario": env_name_map.get(env, env),
                        "Algorithm": algo.upper(),
                        "Min KL Divergence": min_val,
                        "Convergence Iterations": min_idx,
                        "Total Iterations": total_iterations,
                        "Convergence Efficiency": min_idx / total_iterations,
                        "Convergence Rate": convergence_rate,
                        "Stability Metric": stability,
                        "Initial Convergence Speed": early_convergence,
                        "Relative Improvement (%)": relative_improvement
                    })
            except Exception as e:
                print(f"Failed to load {algo} algorithm data in {env} scenario: {e}")
    
    results_df = pd.DataFrame(results)
    return results_df

def generate_extended_latex_table(results_df, selected_metrics):
    """
    Generate a LaTeX three-line table with extended statistics.
    
    Args:
    results_df: DataFrame containing results
    selected_metrics: List of metrics to include in the table
    
    Returns:
    latex_code: LaTeX table code
    """
    # Sort and process data
    results_df = results_df.sort_values(by=["Scenario", "Min KL Divergence"])
    
    # Mark the best algorithm for each scenario
    best_per_scene = results_df.groupby("Scenario")["Min KL Divergence"].idxmin()
    results_df["Is Best"] = False
    results_df.loc[best_per_scene, "Is Best"] = True
    
    # Format values
    format_dict = {
        "Min KL Divergence": lambda x: f"{x:.4f}",
        "Convergence Iterations": lambda x: f"{int(x)}",
        "Total Iterations": lambda x: f"{int(x)}",
        "Convergence Efficiency": lambda x: f"{x:.2f}",
        "Convergence Rate": lambda x: f"{x:.5f}",
        "Stability Metric": lambda x: f"{x:.5f}",
        "Initial Convergence Speed": lambda x: f"{x:.5f}",
        "Relative Improvement (%)": lambda x: f"{x:.1f}"
    }
    
    for metric in selected_metrics:
        if metric in format_dict:
            results_df[metric] = results_df[metric].apply(format_dict[metric])
    
    # Generate table header
    column_count = len(selected_metrics) + 2  # +2 for Scenario and Algorithm
    tabular_format = "c" * column_count
    
    latex_code = f"""
\\begin{{table}}[h]
    \\centering
    \\renewcommand{{\\arraystretch}}{{1.5}}
    \\setlength{{\\tabcolsep}}{{8pt}} % Adjust column width
    \\bicaption{{Convergence Performance Comparison of Different Algorithms in Various Scenarios}}{{Convergence Performance Comparison of Different Algorithms in Various Scenarios}}
    \\label{{tab:convergence_comparison_extended}}
    \\begin{{tabular}}{{{tabular_format}}}
    \\toprule[1.5pt]
    Scenario & Algorithm"""
    
    # Add metric names to header
    for metric in selected_metrics:
        latex_code += f" & {metric}"
    
    latex_code += " \\\\\n    \\midrule[0.75pt]\n"
    
    current_scene = None
    scene_count = 0
    
    for _, row in results_df.iterrows():
        if current_scene != row["Scenario"]:
            if current_scene is not None:
                latex_code += r"    \midrule[0.75pt]" + "\n"
            current_scene = row["Scenario"]
            scene_count = 0
        
        # Bold the best algorithm
        if row["Is Best"]:
            algo_str = r"\textbf{" + row["Algorithm"] + r"}"
        else:
            algo_str = row["Algorithm"]
        
        # Use multirow to merge scenario cells
        if scene_count == 0:
            scene_cell = r"\multirow{4}{*}{" + row['Scenario'] + r"}"
        else:
            scene_cell = ""
        
        latex_code += f"    {scene_cell} & {algo_str}"
        
        for metric in selected_metrics:
            value = row[metric]
            if metric == "Min KL Divergence" and row["Is Best"]:
                value = r"\textbf{" + value + r"}"
            latex_code += f" & {value}"
        
        latex_code += " \\\\\n"
        scene_count += 1
    
    latex_code += r"""    \bottomrule[1.5pt]  
    \end{tabular}
\end{table}
"""
    return latex_code

# Main function
def main():
    env_list = ["merge", "right", "stop"]
    algo_list = ["pso", "nsga3", "age2"]
    
    # Analyze overall convergence performance of algorithms, including extended statistics
    print("Loading data...")
    results_df = load_and_analyze_data_extended(env_list, algo_list)
    
    # Generate tables for different metric combinations
    # Table 1: Basic metrics
    basic_metrics = ["Min KL Divergence", "Convergence Iterations", "Convergence Efficiency"]
    latex_table_basic = generate_extended_latex_table(results_df, basic_metrics)
    
    # Table 2: Convergence speed related metrics
    speed_metrics = ["Min KL Divergence", "Convergence Rate", "Initial Convergence Speed"]
    latex_table_speed = generate_extended_latex_table(results_df, speed_metrics)
    
    # Table 3: Stability and improvement rate
    stability_metrics = ["Min KL Divergence", "Stability Metric", "Relative Improvement (%)"]
    latex_table_stability = generate_extended_latex_table(results_df, stability_metrics)
    
    # Table 4: Comprehensive metrics
    comprehensive_metrics = ["Min KL Divergence", "Convergence Iterations", "Initial Convergence Speed", "Convergence Efficiency"]
    latex_table_comprehensive = generate_extended_latex_table(results_df, comprehensive_metrics)
    
    return latex_table_basic, latex_table_speed, latex_table_stability, latex_table_comprehensive

# Run main function
if __name__ == "__main__":
    latex_table_basic, latex_table_speed, latex_table_stability, latex_table_comprehensive = main()
    print(latex_table_comprehensive)
    print("LaTeX table generated")


## Comparison of Convergence Speed for Multi-objective Algorithm Performance Metrics

In [ ]:
import numpy as np
from task import SUMO_task, pbounds  
import matplotlib.pyplot as plt
import pickle
from multi_object_optimization import MooSUMOProblem, SinSUMOProblem  # Assume these modules exist and are available
import pandas as pd
import itertools

plt.rcParams['font.size'] = 15
plt.rcParams["font.family"] = ["Times New Roman", "SimSun"]
plt.rcParams["font.serif"] = ["Times New Roman", "SimSun"]
plt.rcParams["font.sans-serif"] = ["Times New Roman", "SimSun"]
plt.rcParams["mathtext.fontset"] = "custom"

SAVED_PATH = "../output/plot/iteration"


def plot_object_f(env, algo):
    # English label mapping
    name_map = {
        'Car acc': 'Car Acceleration',
        'Car dhw': 'Car Headway',
        'Car v': 'Car Speed',
        'Bus acc': 'Bus Acceleration',
        'Bus dhw': 'Bus Headway',
        'Bus v': 'Bus Speed'
    }

    markers = itertools.cycle(('^', 'x', 's', '*', 'h', 'H', 'D', 'd'))
    marker_size = 6
    line_styles = itertools.cycle((':', '--', '-.', '-'))
    plt.figure(figsize=(7.5, 6))  # Adjust figure size
    fig, ax = plt.subplots()

    try:
        with open(f'../output/data_cache/{env}_{algo}.pkl', 'rb') as f:
            res = pickle.load(f)
    except FileNotFoundError:
        print(f"File not found: ../output/data_cache/{env}_{algo}.pkl")
        return
    
    history = res.history
    F_list = [entry.pop.get('F') for entry in history]
    all_F = np.vstack(F_list)
    target_names = ['Car acc', 'Car dhw', 'Car v', 'Bus acc', 'Bus dhw', 'Bus v']
    df = pd.DataFrame(all_F, columns=target_names)

    lines = []  # Store line objects for later color retrieval
    min_points = []  # Store the coordinates and labels of the minimum value for each metric

    for target in target_names:
        turning_points = df[target].cummin()
        line, = plt.plot(df.index, turning_points, label=name_map.get(target, target),
                         marker=next(markers),
                         markersize=marker_size,
                         markerfacecolor='none',
                         markevery=[i for i in range(1, len(turning_points) - 1)
                                    if turning_points[i] != turning_points[i - 1]
                                    or turning_points[i] != turning_points[i + 1]],
                         linewidth=0.8)
        lines.append(line)

        # Find the minimum value point and add it to the min_points list
        min_idx = turning_points.idxmin()
        min_val = turning_points.min()
        min_points.append((min_idx, min_val, name_map.get(target, target)))

    global_min = min(min_points, key=lambda x: x[1])

    # Add annotations and arrows
    for i, (x, y, label) in enumerate(min_points):
        arrow_color = lines[i].get_color() # Get the color of the corresponding curve

        hight = [30,40,60,80,100, 120]
        if (x, y) == (global_min[0], global_min[1]):
            plt.annotate(f'{y:.3f}',
                        xy=(x, y), xycoords='data',
                        xytext=(-15,hight[i]), textcoords='offset points',  # Adjust annotation position
                        arrowprops=dict(arrowstyle="->", color=arrow_color, lw=1, shrinkB=15),
                        ha='center', va='bottom', fontweight='bold')
        else:
            plt.annotate(f'{y:.3f}',
                        xy=(x, y), xycoords='data',
                        xytext=(-15,hight[i]), textcoords='offset points',  # Adjust annotation position
                        arrowprops=dict(arrowstyle="->", color=arrow_color, lw=1, shrinkB=15),
                        ha='center', va='bottom')

    plt.ylim(-0.05, 1)  # Adjust y-axis
    plt.xlim(-100, 3100)
    plt.legend(loc='upper right',fontsize=13,ncol=2)  # Add legend

    # Set title according to environment
    title_map = {
        "merge": "MERGE",
        "stop": "STOP",
        "right": "RIGHT"
    }
    title_en = title_map.get(env, env.upper())  # Get English title, use uppercase if not found

    plt.title(f'MoP Comparison for {algo.upper()} in {title_en} Scenario')  # Use English title
    plt.xlabel('Iteration')
    plt.ylabel('Minimum KL Divergence')

    plt.tight_layout(pad=2)  # Adjust layout
    plt.grid(visible=True, linestyle='--', alpha=0.5)

    plt.savefig(f'{SAVED_PATH}/{env}_{algo}_object_f.pdf', dpi=300)
    plt.show()


algo_list = ["nsga3", "age2"]
envs = ["merge", "stop", "right"]
for env in envs:
    for algo in algo_list:
        plot_object_f(env, algo)

## Table of Minimum KL Divergence Values for Each Performance Metric under Different Scenarios and Algorithms

In [ ]:
import numpy as np
import pickle
import pandas as pd
import os

def extract_data_and_generate_latex():
    envs = ["merge", "stop", "right"]
    algo_list = ["nsga3", "age2"]
    
    # Chinese label mapping
    name_map = {
        'Car acc': 'Car Acceleration',
        'Car dhw': 'Car Headway',
        'Car v': 'Car Velocity',
        'Bus acc': 'Bus Acceleration',
        'Bus dhw': 'Bus Headway',
        'Bus v': 'Bus Velocity'
    }
    
    # English label mapping
    eng_map = {
        'Car acc': 'Car Acceleration',
        'Car dhw': 'Car Headway',
        'Car v': 'Car Velocity',
        'Bus acc': 'Bus Acceleration',
        'Bus dhw': 'Bus Headway',
        'Bus v': 'Bus Velocity'
    }
    
    # Scenario mapping
    env_map = {
        "merge": "MERGE",
        "stop": "STOP",
        "right": "NARROW"
    }
    
    # Algorithm mapping
    algo_map = {
        "nsga3": "NSGA-III",
        "age2": "AGE-II"
    }
    
    # Create result storage structure
    results = {}
    target_names = ['Car acc', 'Car dhw', 'Car v', 'Bus acc', 'Bus dhw', 'Bus v']
    
    # Extract data
    for env in envs:
        results[env] = {}
        for algo in algo_list:
            try:
                with open(f'../output/data_cache/{env}_{algo}.pkl', 'rb') as f:
                    res = pickle.load(f)
                    
                history = res.history
                F_list = [entry.pop.get('F') for entry in history]
                all_F = np.vstack(F_list)
                df = pd.DataFrame(all_F, columns=target_names)
                
                # Calculate the minimum value for each metric
                min_values = {}
                for target in target_names:
                    turning_points = df[target].cummin()
                    min_val = turning_points.min()
                    min_values[target] = min_val
                
                results[env][algo] = min_values
                
            except FileNotFoundError:
                print(f"File not found: ../output/data_cache/{env}_{algo}.pkl")
                results[env][algo] = {target: "N/A" for target in target_names}
    
    # Find the global minimum for each metric
    global_min = {}
    for target in target_names:
        min_val = float('inf')
        for env in envs:
            for algo in algo_list:
                if isinstance(results[env][algo][target], (int, float)) and results[env][algo][target] < min_val:
                    min_val = results[env][algo][target]
        global_min[target] = min_val
    
    # Generate LaTeX table
    latex_code = r'''\begin{table}[h]
    \centering
    \renewcommand{\arraystretch}{1.5}
    \setlength{\tabcolsep}{10pt} % Adjust column width
    \bicaption{Minimum KL Divergence Values for Different Scenarios and Algorithms}{Minimum KL Divergence Values for Different Scenarios and Algorithms}
    \label{tab:kl_divergence}
    \begin{tabular}{ccccccccc}
    \toprule[1.5pt]
    \multirow{2}{*}{Scenario} & \multirow{2}{*}{Algorithm} & \multicolumn{3}{c}{Car} & \multicolumn{3}{c}{Bus} \\
    \cmidrule(lr){3-5} \cmidrule(lr){6-8}
    & & Acceleration & Headway & Velocity & Acceleration & Headway & Velocity \\
    \midrule[0.75pt]'''
    
    # Add data rows
    for env in envs:
        for i, algo in enumerate(algo_list):
            if i == 0:
                row = f"\n    \\multirow{{2}}{{*}}{{{env_map[env]}}} & {algo_map[algo]}"
            else:
                row = f"\n    & {algo_map[algo]}"
                
            for target in ['Car acc', 'Car dhw', 'Car v', 'Bus acc', 'Bus dhw', 'Bus v']:
                if isinstance(results[env][algo][target], (int, float)):
                    # Bold if it is the minimum value
                    if abs(results[env][algo][target] - global_min[target]) < 1e-6:  # Use small epsilon for float comparison
                        row += f" & \\textbf{{{results[env][algo][target]:.3f}}}"
                    else:
                        row += f" & {results[env][algo][target]:.3f}"
                else:
                    row += f" & {results[env][algo][target]}"
            
            latex_code += row + " \\\\"
            
        if env != envs[-1]:
            latex_code += "\n    \\midrule"
    
    # Finish table
    latex_code += r'''
    \bottomrule[1.5pt]  
    \end{tabular}
\end{table}'''
    
    return latex_code

# Run the function and print the result
latex_table = extract_data_and_generate_latex()
print(latex_table)


## Comparison of Distributions: Calibrated Parameters, Real Data, and Uncalibrated SUMO Parameters

In [ ]:
# Comparison of Distributions: Calibrated Parameters, Real Data, and Uncalibrated SUMO Parameters

import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FormatStrFormatter

plt.rcParams["font.family"] = ["Times New Roman", "SimSun"]
plt.rcParams["font.serif"] = ["Times New Roman", "SimSun"]
plt.rcParams["font.sans-serif"] = ["Times New Roman", "SimSun"]
plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["mathtext.rm"] = "SimSun"
SAVED_PATH = "../output/plot/distribution"

# Vehicle type mapping
vehicle_type_map = {
    "bus": "Bus",
    "car": "Car"
}

# Variable name mapping
name_map = {
    "velocity": "Velocity",
    "acceleration": "Acceleration",
    "time headway": "Time Headway"
}

def plot_distribution_comparison(data_list, vehicle_type, variable, output_dir, index):
    """
    Plot distribution comparison (histogram + kernel density estimation)
    """
    plt.figure(figsize=(3.6, 3))
    colors = ["#5b9bd5", "#ed7d31", "#70ad47"]
    labels = ["Calibrated", "Ground Truth", "SUMO"]

    for idx, (data, color) in enumerate(zip(data_list, colors)):
        hist_data, bin_width, bin_centers, kde_x, kde_y = data
        plt.bar(
            bin_centers,
            height=hist_data,
            width=bin_width,
            align="center",
            edgecolor="k",
            alpha=0.39,
            color=color,
            linewidth=0.3,
            label=f"{labels[idx]}",
        )
        plt.plot(kde_x, kde_y, color=color, linestyle="-", linewidth=0.5)

    if len(data_list) > 1:
        _, _, _, kde_x_second, _ = data_list[1]
        plt.xlim(min(kde_x_second), max(kde_x_second))

    subplot_label = f"({chr(97 + index)})"
    plt.text(-0.1, 1.03, subplot_label, transform=plt.gca().transAxes, fontsize=12)

    font_size = 12

    vehicle_type_en = vehicle_type_map.get(vehicle_type.lower(), vehicle_type)

    if variable == "xVelocity":
        plt.xlabel(f"Velocity $(m/s)$", fontsize=font_size, labelpad=1)
        variable_en = name_map.get("velocity", "Velocity")
    elif variable == "xAcceleration":
        plt.xlabel(f"Acceleration $(m/s^2)$", fontsize=font_size, labelpad=1)
        variable_en = name_map.get("acceleration", "Acceleration")
    elif variable == "dhw":
        plt.xlabel(f"Time Headway $(m)$", fontsize=font_size, labelpad=1)
        variable_en = name_map.get("time headway", "Time Headway")
    else:
        variable_en = variable

    plt.title(f"{vehicle_type_en} {variable_en} Distribution", fontsize=font_size)
    plt.legend(loc="upper right", fontsize=12)
    plt.xticks(fontsize=font_size)
    plt.yticks(fontsize=font_size)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    plt.grid(visible=True, linestyle='--', alpha=0.6)
    plt.tight_layout(pad=0.2)

    output_file = os.path.join(
        output_dir, f"{env}_{vehicle_type}_{variable}_distribution.pdf"
    )
    plt.savefig(output_file, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.show()

def batch_plot_distribution_comparison(caches, output_dir, start_index=0):
    """
    Batch plot distribution comparisons from cache files
    """
    dfs = []
    for cache in caches:
        with open(cache, "rb") as f:
            dfs.append(pickle.load(f))

    hist_kde_data = [df["hist_kde_data"] for df in dfs]

    keys = set(hist_kde_data[0].keys())
    for data in hist_kde_data[1:]:
        keys.intersection_update(data.keys())

    idx = start_index
    for cache_key in sorted(list(keys)):
        vehicle_type, variable = cache_key.split("_")
        data_list = [data[cache_key] for data in hist_kde_data]
        plot_distribution_comparison(data_list, vehicle_type, variable, output_dir, idx)
        idx += 1
    return idx

current_index = 0
for env in ["merge", "right", "stop"]:
    current_index = batch_plot_distribution_comparison([
        f"../output/data_raw/{env}/_cache.pkl",
        f"../output/data_cache/{env}_cache.pkl",
        f"../output/data_raw/{env}_origin/_cache.pkl"
    ], SAVED_PATH, current_index)


## Pareto Front Distribution Plot

In [ ]:
# Pareto Front Distribution Plot (English Version)

from multi_object_optimization import MooSUMOProblem, SinSUMOProblem

import pickle
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Font settings for English and Chinese
plt.rcParams['font.size'] = 13
plt.rcParams['font.family'] = ['Times New Roman', 'SimSun']
plt.rcParams['font.weight'] = 'light'
plt.rcParams['axes.unicode_minus'] = False

# List of environments and algorithms
environments = ["merge", "stop", "right"]
algorithms = ["age2", "nsga3"]

# Mapping from environment key to Chinese name
env_map = {
    "merge": "MERGE",
    "stop": "STOP",
    "right": "NARROW"
}

# Feature name mapping
feature_names = {
    0: "Acceleration",
    1: "Headway",
    2: "Speed"
}

# Indices for car and bus features
car_indices = [0, 1, 2]  # Car: Acceleration, Headway, Speed
bus_indices = [3, 4, 5]  # Bus: Acceleration, Headway, Speed

for env in environments:
    for alg in algorithms:
        try:
            with open(f'../output/data_cache/{env}_{alg}.pkl', 'rb') as f:
                res = pickle.load(f)
                F = res.F

                # Find ideal points for car and bus
                ideal_point_index_car = np.argmin(np.sum(F[:, car_indices], axis=1))
                ideal_point_car = F[ideal_point_index_car, car_indices]
                
                ideal_point_index_bus = np.argmin(np.sum(F[:, bus_indices], axis=1))
                ideal_point_bus = F[ideal_point_index_bus, bus_indices]

                fig = plt.figure(figsize=(20, 5.5))
                
                env_en = env_map.get(env, env.upper())
                
                # 3D scatter plot for Pareto front
                ax_3d = fig.add_subplot(141, projection='3d')
                ax_3d.set_title(f'{env_en} {alg.upper()} Pareto Front')
                ax_3d.set_xlabel('Acceleration', labelpad=5, rotation=-10)
                ax_3d.set_ylabel('Headway', labelpad=5, rotation=50)
                ax_3d.set_zlabel('Speed', labelpad=5, rotation=90)

                # Car data
                ax_3d.scatter(F[:, car_indices[0]], F[:, car_indices[1]], F[:, car_indices[2]], 
                             s=30, facecolors='none', edgecolors='blue', label='Car')

                # Bus data
                ax_3d.scatter(F[:, bus_indices[0]], F[:, bus_indices[1]], F[:, bus_indices[2]], 
                             s=30, facecolors='none', edgecolors='green', label='Bus')

                # Mark ideal points for car and bus
                ax_3d.scatter(ideal_point_car[0], ideal_point_car[1], ideal_point_car[2], 
                             s=100, c='red', marker='x', label='Ideal Point (Car)', facecolors='white')
                ax_3d.scatter(ideal_point_bus[0], ideal_point_bus[1], ideal_point_bus[2], 
                             s=100, c='red', marker='o', label='Ideal Point (Bus)', facecolors='white')
                
                # 2D feature index pairs: (Acceleration, Headway), (Acceleration, Speed), (Headway, Speed)
                pairs = [(0, 1), (0, 2), (1, 2)]

                for i, (x_idx, y_idx) in enumerate(pairs):
                    ax = fig.add_subplot(1, 4, i + 2)
                    ax.set_title(f'{env_en} {alg.upper()}: {feature_names[x_idx]} vs {feature_names[y_idx]}')
                    ax.set_xlabel(f'{feature_names[x_idx]} KL Divergence')
                    ax.set_ylabel(f'{feature_names[y_idx]} KL Divergence')

                    # Car data
                    ax.scatter(F[:, car_indices[x_idx]], F[:, car_indices[y_idx]], 
                              s=40, facecolors='none', edgecolors='blue', label='Car', alpha=0.6)

                    # Bus data
                    ax.scatter(F[:, bus_indices[x_idx]], F[:, bus_indices[y_idx]], 
                              s=40, facecolors='none', edgecolors='green', label='Bus', alpha=0.6)
                    ax.grid(visible=True, linestyle='--', alpha=0.5)

                    # Mark ideal points for car and bus
                    ax.scatter(ideal_point_car[x_idx], ideal_point_car[y_idx], 
                              s=100, c='red', marker='x', label='Ideal Point (Car)')
                    ax.scatter(ideal_point_bus[x_idx], ideal_point_bus[y_idx], 
                              s=100, c='red', marker='o', label='Ideal Point (Bus)')
                    
                    # Annotate ideal points
                    ax.annotate(f'({ideal_point_car[x_idx]:.3f}, {ideal_point_car[y_idx]:.3f})',
                               xy=(ideal_point_car[x_idx], ideal_point_car[y_idx]),
                               xytext=(10, 60), textcoords='offset points',
                               arrowprops=dict(arrowstyle="->", color='black', linestyle='--', lw=2),
                               color='black', fontweight='bold')
                    
                    ax.annotate(f'({ideal_point_bus[x_idx]:.3f}, {ideal_point_bus[y_idx]:.3f})',
                               xy=(ideal_point_bus[x_idx], ideal_point_bus[y_idx]),
                               xytext=(90, 30), textcoords='offset points',
                               arrowprops=dict(arrowstyle="->", color='black', linestyle='--', lw=1),
                               color='black')
                    
                    # Show legend only in the first subplot to avoid repetition
                    ax.legend(fontsize=13, loc="upper right")

                plt.tight_layout(pad=2.5)
                plt.savefig(f'../output/plot/scatter/{env}_{alg}_pareto_front.pdf', dpi=300)
                plt.show()
                
        except Exception as e:
            print(e)


## Pareto Front Table

In [ ]:
from multi_object_optimization import MooSUMOProblem, SinSUMOProblem
import pickle
import numpy as np
import os
import pandas as pd

# List of environments and algorithms
environments = ["merge", "stop", "right"]
algorithms = ["age2", "nsga3"]

# Mapping for environment names (Chinese to English)
env_map = {
    "merge": "MERGE",
    "stop": "STOP",
    "right": "NARROW"
}

# Feature names mapping (Chinese to English)
feature_names = {
    0: "Acceleration",
    1: "Headway",
    2: "Speed"
}

# Indices for car and bus features
car_indices = [0, 1, 2]  # Car: Acceleration, Headway, Speed
bus_indices = [3, 4, 5]  # Bus: Acceleration, Headway, Speed

results = []

# Iterate over all environment and algorithm combinations
for env in environments:
    for alg in algorithms:
        try:
            with open(f'../output/data_cache/{env}_{alg}.pkl', 'rb') as f:
                res = pickle.load(f)
                F = res.F

                # Find ideal points for car and bus (minimize sum of objectives)
                ideal_point_index_car = np.argmin(np.sum(F[:, car_indices], axis=1))
                ideal_point_car = F[ideal_point_index_car, car_indices]
                
                ideal_point_index_bus = np.argmin(np.sum(F[:, bus_indices], axis=1))
                ideal_point_bus = F[ideal_point_index_bus, bus_indices]
                
                pareto_points_count = F.shape[0]
                
                # Compute statistics for each feature
                car_stats = {
                    "Acceleration": {
                        "min": np.min(F[:, car_indices[0]]),
                        "max": np.max(F[:, car_indices[0]]),
                        "mean": np.mean(F[:, car_indices[0]]),
                        "ideal": ideal_point_car[0]
                    },
                    "Headway": {
                        "min": np.min(F[:, car_indices[1]]),
                        "max": np.max(F[:, car_indices[1]]),
                        "mean": np.mean(F[:, car_indices[1]]),
                        "ideal": ideal_point_car[1]
                    },
                    "Speed": {
                        "min": np.min(F[:, car_indices[2]]),
                        "max": np.max(F[:, car_indices[2]]),
                        "mean": np.mean(F[:, car_indices[2]]),
                        "ideal": ideal_point_car[2]
                    }
                }
                
                bus_stats = {
                    "Acceleration": {
                        "min": np.min(F[:, bus_indices[0]]),
                        "max": np.max(F[:, bus_indices[0]]),
                        "mean": np.mean(F[:, bus_indices[0]]),
                        "ideal": ideal_point_bus[0]
                    },
                    "Headway": {
                        "min": np.min(F[:, bus_indices[1]]),
                        "max": np.max(F[:, bus_indices[1]]),
                        "mean": np.mean(F[:, bus_indices[1]]),
                        "ideal": ideal_point_bus[1]
                    },
                    "Speed": {
                        "min": np.min(F[:, bus_indices[2]]),
                        "max": np.max(F[:, bus_indices[2]]),
                        "mean": np.mean(F[:, bus_indices[2]]),
                        "ideal": ideal_point_bus[2]
                    }
                }
                
                # Store results
                results.append({
                    "Environment": env_map[env],
                    "Algorithm": alg.upper(),
                    "Pareto Point Count": pareto_points_count,
                    "Car Stats": car_stats,
                    "Bus Stats": bus_stats
                })
                
        except Exception as e:
            print(f"Error processing {env}_{alg}: {e}")
            continue

# Generate LaTeX table for ideal points
def generate_latex_table(results):
    latex = """
\\begin{table}[htbp]
  \\centering
  \\caption{Pareto Front Statistics for Multi-objective Optimization Algorithms in Different Scenarios}
  \\label{tab:pareto_front_stats}
  \\begin{tabular}{ccccccccc}
    \\toprule
    \\multirow{2}{*}{Scenario} & \\multirow{2}{*}{Algorithm} & \\multirow{2}{*}{Pareto Points} & \\multicolumn{3}{c}{Car Ideal Point} & \\multicolumn{3}{c}{Bus Ideal Point} \\\\
    \\cmidrule(lr){4-6} \\cmidrule(lr){7-9}
    & & & Acceleration & Headway & Speed & Acceleration & Headway & Speed \\\\
    \\midrule
"""
    for res in results:
        latex += f"    {res['Environment']} & {res['Algorithm']} & {res['Pareto Point Count']} & "
        latex += f"{res['Car Stats']['Acceleration']['ideal']:.3f} & {res['Car Stats']['Headway']['ideal']:.3f} & {res['Car Stats']['Speed']['ideal']:.3f} & "
        latex += f"{res['Bus Stats']['Acceleration']['ideal']:.3f} & {res['Bus Stats']['Headway']['ideal']:.3f} & {res['Bus Stats']['Speed']['ideal']:.3f} \\\\\n"
    latex += """    \\bottomrule
  \\end{tabular}
\\end{table}
"""
    return latex

# Generate LaTeX table for value ranges
def generate_range_table(results):
    latex = """
\\begin{table}[htbp]
  \\centering
  \\caption{Pareto Front Feature Ranges for Multi-objective Optimization Algorithms in Different Scenarios}
  \\label{tab:pareto_front_ranges}
  \\begin{tabular}{cccccccc}
    \\toprule
    \\multirow{2}{*}{Scenario} & \\multirow{2}{*}{Algorithm} & \\multicolumn{3}{c}{Car Feature Range (min-max)} & \\multicolumn{3}{c}{Bus Feature Range (min-max)} \\\\
    \\cmidrule(lr){3-5} \\cmidrule(lr){6-8}
    & & Acceleration & Headway & Speed & Acceleration & Headway & Speed \\\\
    \\midrule
"""
    for res in results:
        latex += f"    {res['Environment']} & {res['Algorithm']} & "
        latex += f"{res['Car Stats']['Acceleration']['min']:.3f}-{res['Car Stats']['Acceleration']['max']:.3f} & "
        latex += f"{res['Car Stats']['Headway']['min']:.3f}-{res['Car Stats']['Headway']['max']:.3f} & "
        latex += f"{res['Car Stats']['Speed']['min']:.3f}-{res['Car Stats']['Speed']['max']:.3f} & "
        latex += f"{res['Bus Stats']['Acceleration']['min']:.3f}-{res['Bus Stats']['Acceleration']['max']:.3f} & "
        latex += f"{res['Bus Stats']['Headway']['min']:.3f}-{res['Bus Stats']['Headway']['max']:.3f} & "
        latex += f"{res['Bus Stats']['Speed']['min']:.3f}-{res['Bus Stats']['Speed']['max']:.3f} \\\\\n"
    latex += """    \\bottomrule
  \\end{tabular}
\\end{table}
"""
    return latex

# Generate LaTeX table for means
def generate_mean_table(results):
    latex = """
\\begin{table}[htbp]
  \\centering
  \\caption{Pareto Front Feature Means for Multi-objective Optimization Algorithms in Different Scenarios}
  \\label{tab:pareto_front_means}
  \\begin{tabular}{cccccccc}
    \\toprule
    \\multirow{2}{*}{Scenario} & \\multirow{2}{*}{Algorithm} & \\multicolumn{3}{c}{Car Feature Mean} & \\multicolumn{3}{c}{Bus Feature Mean} \\\\
    \\cmidrule(lr){3-5} \\cmidrule(lr){6-8}
    & & Acceleration & Headway & Speed & Acceleration & Headway & Speed \\\\
    \\midrule
"""
    for res in results:
        latex += f"    {res['Environment']} & {res['Algorithm']} & "
        latex += f"{res['Car Stats']['Acceleration']['mean']:.3f} & {res['Car Stats']['Headway']['mean']:.3f} & {res['Car Stats']['Speed']['mean']:.3f} & "
        latex += f"{res['Bus Stats']['Acceleration']['mean']:.3f} & {res['Bus Stats']['Headway']['mean']:.3f} & {res['Bus Stats']['Speed']['mean']:.3f} \\\\\n"
    latex += """    \\bottomrule
  \\end{tabular}
\\end{table}
"""
    return latex

# Generate comprehensive LaTeX table
def generate_comprehensive_table(results):
    latex = """
\\begin{table}[htbp]
  \\centering
  \\caption{Comprehensive Pareto Front Statistics for Multi-objective Optimization Algorithms in Different Scenarios}
  \\label{tab:pareto_front_comprehensive}
  \\resizebox{\\textwidth}{!}{%
  \\begin{tabular}{ccc|ccc|ccc|ccc|ccc}
    \\toprule
    \\multirow{3}{*}{Scenario} & \\multirow{3}{*}{Algorithm} & \\multirow{3}{*}{Pareto Points} & 
    \\multicolumn{6}{c|}{Car} & \\multicolumn{6}{c}{Bus} \\\\
    \\cmidrule(lr){4-9} \\cmidrule(lr){10-15}
    & & & \\multicolumn{3}{c|}{Feature Range (min-max)} & \\multicolumn{3}{c|}{Ideal Point} & 
    \\multicolumn{3}{c|}{Feature Range (min-max)} & \\multicolumn{3}{c}{Ideal Point} \\\\
    \\cmidrule(lr){4-6} \\cmidrule(lr){7-9} \\cmidrule(lr){10-12} \\cmidrule(lr){13-15}
    & & & Acceleration & Headway & Speed & Acceleration & Headway & Speed & Acceleration & Headway & Speed & Acceleration & Headway & Speed \\\\
    \\midrule
"""
    for res in results:
        latex += f"    {res['Environment']} & {res['Algorithm']} & {res['Pareto Point Count']} & "
        # Car feature range
        latex += f"{res['Car Stats']['Acceleration']['min']:.3f}-{res['Car Stats']['Acceleration']['max']:.3f} & "
        latex += f"{res['Car Stats']['Headway']['min']:.3f}-{res['Car Stats']['Headway']['max']:.3f} & "
        latex += f"{res['Car Stats']['Speed']['min']:.3f}-{res['Car Stats']['Speed']['max']:.3f} & "
        # Car ideal point
        latex += f"{res['Car Stats']['Acceleration']['ideal']:.3f} & {res['Car Stats']['Headway']['ideal']:.3f} & {res['Car Stats']['Speed']['ideal']:.3f} & "
        # Bus feature range
        latex += f"{res['Bus Stats']['Acceleration']['min']:.3f}-{res['Bus Stats']['Acceleration']['max']:.3f} & "
        latex += f"{res['Bus Stats']['Headway']['min']:.3f}-{res['Bus Stats']['Headway']['max']:.3f} & "
        latex += f"{res['Bus Stats']['Speed']['min']:.3f}-{res['Bus Stats']['Speed']['max']:.3f} & "
        # Bus ideal point
        latex += f"{res['Bus Stats']['Acceleration']['ideal']:.3f} & {res['Bus Stats']['Headway']['ideal']:.3f} & {res['Bus Stats']['Speed']['ideal']:.3f} \\\\\n"
    latex += """    \\bottomrule
  \\end{tabular}%
  }
\\end{table}
"""
    return latex

# Output LaTeX tables
ideal_point_table = generate_latex_table(results)
range_table = generate_range_table(results)
mean_table = generate_mean_table(results)
comprehensive_table = generate_comprehensive_table(results)

print("\nIdeal Point Table:")
print(ideal_point_table)

# Uncomment below to print other tables
# print("\nFeature Range Table:")
# print(range_table)
# print("\nFeature Mean Table:")
# print(mean_table)
# print("\nComprehensive Table:")
# print(comprehensive_table)

# Uncomment below to save all tables to a file
# with open("pareto_front_tables.tex", "w", encoding="utf-8") as f:
#     f.write("% Ideal Point Table\n")
#     f.write(ideal_point_table)
#     f.write("\n\n% Feature Range Table\n")
#     f.write(range_table)
#     f.write("\n\n% Feature Mean Table\n")
#     f.write(mean_table)
#     f.write("\n\n% Comprehensive Table\n")
#     f.write(comprehensive_table)
